In [10]:
import requests
import pandas as pd
import time

# User Inputs
COINS = [ "BANANAS31", "MUBARAK", "BTC"]    # Binance symbols (e.g., BTC, BANANA, BROCCOLI)
START_DATE = "2025-04-01"           # YYYY-MM-DD
END_DATE = "2025-04-10"             # YYYY-MM-DD
INTERVAL = "5m"                     # 1m, 5m, 15m, 1h, 1d

# Convert dates to timestamps (Binance uses milliseconds)
start_ts = int(pd.to_datetime(START_DATE).timestamp() * 1000)
end_ts = int(pd.to_datetime(END_DATE).timestamp() * 1000)

# API endpoints
urls = [
    "https://api.binance.com/api/v3/klines",  # Main Binance API
    "https://api.binance.us/api/v3/klines"    # Fallback for US users
]
data_dict = {}

for coin in COINS:
    pair = f"{coin}USDT"
    if coin not in ["BTC", "ETH", "BNB", "DOGE", "SHIB", "BANANA", "BROCCOLI"]:
        print(f"Warning: {coin} may not be listed on Binance")

    success = False
    for url in urls:  # Try main API, then fallback
        for attempt in range(3):
            try:
                response = requests.get(url, params={
                    "symbol": pair,
                    "interval": INTERVAL,
                    "startTime": start_ts,
                    "endTime": end_ts,
                    "limit": 1000
                }, timeout=5)

                if response.status_code == 200:
                    data = response.json()
                    if not data:
                        print(f"No data for {pair} on {url}")
                        break
                    df = pd.DataFrame(data, columns=[
                        "time", "open", "high", "low", "close", "volume",
                        "close_time", "quote_volume", "trades", "tb_base", "tb_quote", "ignore"
                    ])
                    df = df[["time", "close"]].assign(
                        time=pd.to_datetime(df["time"], unit="ms"),
                        close=df["close"].astype(float)
                    )
                    data_dict[coin] = df.set_index("time")["close"].rename(f"{coin}_close")
                    success = True
                    break
                elif response.status_code == 451:
                    print(f"Attempt {attempt + 1} failed for {pair} on {url}: Geo-restriction (451)")
                    break  # Move to next URL
                else:
                    print(f"Attempt {attempt + 1} failed for {pair} on {url}: API error ({response.status_code})")
            except requests.RequestException as e:
                print(f"Attempt {attempt + 1} failed for {pair} on {url}: Connection error ({str(e)})")
            time.sleep(1)
        if success:
            break
    if not success:
        print(f"Skipping {pair}: Failed on all endpoints")

# Save to Excel
if data_dict:
    df = pd.concat(data_dict.values(), axis=1, join="inner").reset_index()
    df.columns = ["time"] + [f"{coin}_close" for coin in data_dict.keys()]
    print("Preview:")
    print(df.head())
    df.to_excel("binance_coins.xlsx", index=False, sheet_name="Coin_Data")
    print("Saved to binance_coins.xlsx")
else:
    print("No data fetched")

Attempt 1 failed for BANANAS31USDT on https://api.binance.com/api/v3/klines: Geo-restriction (451)
Attempt 1 failed for BANANAS31USDT on https://api.binance.us/api/v3/klines: API error (400)
Attempt 2 failed for BANANAS31USDT on https://api.binance.us/api/v3/klines: API error (400)
Attempt 3 failed for BANANAS31USDT on https://api.binance.us/api/v3/klines: API error (400)
Skipping BANANAS31USDT: Failed on all endpoints
Attempt 1 failed for MUBARAKUSDT on https://api.binance.com/api/v3/klines: Geo-restriction (451)
Attempt 1 failed for MUBARAKUSDT on https://api.binance.us/api/v3/klines: API error (400)
Attempt 2 failed for MUBARAKUSDT on https://api.binance.us/api/v3/klines: API error (400)
Attempt 3 failed for MUBARAKUSDT on https://api.binance.us/api/v3/klines: API error (400)
Skipping MUBARAKUSDT: Failed on all endpoints
Attempt 1 failed for BTCUSDT on https://api.binance.com/api/v3/klines: Geo-restriction (451)
Preview:
                 time  BTC_close
0 2025-04-01 00:00:00   82629